In [50]:
import pandas as pd
import numpy as np

def onoff_dict(arr_raw, return_duration=False, return_transitions = False, return_all = False, treatasone=True):
    #TODO the variable transi is better named sequence
    if not isinstance(arr_raw, list):
        arr_raw = [arr_raw]

    arr_onoff = {}
    arr_transi =  []
    arr_onset =  []
    arr_onnext =  []
    arr_dur =  []
    total_dur = 0
    for i,a in enumerate(arr_raw):
        if isinstance(a, pd.Series) or isinstance(a, pd.DataFrame):
            a = a.values
        arr_s = a[1:]
        arr = a[:-1]

        bool_diff = arr != arr_s
        transi = np.append(arr[bool_diff], arr[-1])
        onset = np.append([0],np.where(bool_diff)[0]+1)
        onnext = np.append(np.array((onset)[1:]), [len(arr)+1])
        dur = onnext-onset
        arr_transi.append(transi)
        arr_onset.append(onset)
        arr_onnext.append(onnext)
        arr_dur.append(dur)

        if treatasone:
            if i > 0:
                total_dur += arr_onnext[i-1][-1]

        for b in np.unique(a):
            b_idx = transi == b
            b_onoff = list(zip(onset[b_idx]+total_dur, dur[b_idx]))
            if b in arr_onoff.keys():
                arr_onoff[b] = arr_onoff[b]+b_onoff
            else:
                arr_onoff[b] = b_onoff
    if treatasone:
        arr_dur = np.concatenate(arr_dur)
        arr_transi = np.concatenate(arr_transi)
        arr_onset = np.concatenate([a+arr_onnext[i-1][-1] if i > 0 else a for i,a in enumerate(arr_onset)])
        arr_onnext = np.concatenate([a+arr_onnext[i-1][-1] if i > 0 else a for i,a in enumerate(arr_onnext)])
        # might hav to work here on further, change how arr_onset and arr_onnext are daved

    if return_all == True:
        return arr_onoff, arr_dur, arr_transi, arr_onset, arr_onnext
    elif return_transitions == True and return_duration == True:
        return arr_onoff, arr_dur, arr_transi
    elif return_duration == True or return_transitions == True:
        if return_duration == True:
            return arr_onoff, arr_dur
        if return_transitions == True:
            return arr_onoff, arr_transi
    else:
        return arr_onoff

In [51]:
etho = np.array([5,5,5,3,3,3,3,3,3,3,2,2,2,1,1,1,1,1,1,3,3,3,3,3,4,4,4,5,5,5,5,5,5,5,5])
arr_onoff, arr_dur, arr_transi, arr_onset, arr_onnext = onoff_dict(etho, return_all=True)
arr_onoff, arr_dur, arr_transi, arr_onset, arr_onnext

({1: [(13, 6)],
  2: [(10, 3)],
  3: [(3, 7), (19, 5)],
  4: [(24, 3)],
  5: [(0, 3), (27, 8)]},
 array([3, 7, 3, 6, 5, 3, 8]),
 array([5, 3, 2, 1, 3, 4, 5]),
 array([ 0,  3, 10, 13, 19, 24, 27]),
 array([ 3, 10, 13, 19, 24, 27, 35]))

In [23]:
arr_dur.shape, arr_transi.shape, arr_onset.shape, arr_onnext.shape

((7,), (7,), (7,), (7,))

In [28]:
arr = etho[:-1]
arr_s = etho[1:]
np.concatenate([[0],np.where([arr != arr_s])[1]+1])

array([ 0,  3, 10, 13, 19, 24, 27])

In [49]:
onset = np.append([0],np.where(arr != arr_s)[0]+1)
onnext = np.append(np.array((onset)[1:]), [len(arr)+1])
onnext-onset

array([3, 7, 3, 6, 5, 3, 8])